In [301]:
import numpy as np
import sympy as sp
import pandas as pd
import os
from sympy.parsing.sympy_parser import parse_expr
import control

In [302]:
# Parameters
r, Mb, rb, Ib, Mw, Iw, g, tau = sp.symbols('r M_b r_b I_b M_w I_w g tau')

In [303]:
subs_dict = {
    Mb: 0.366,    # Body Mass
    rb: 0.09,    # Pendulum center of mass
    Ib: 0.0082,   # Body Inertia
    Mw: 0.409 * 2,    # Wheel mass
    Iw: 0.00029,   # Wheel inertia
    r: 0.06,    # Wheel radius
    g: 9.81     # Gravity
}

# Linear Controller Design

State Space Function(Continuous and Discrete)

In [304]:
# Import linearied A and B
with open('../dynamics/output/A.txt', 'r') as f:
    A = parse_expr(f.read())

with open('../dynamics/output/B.txt', 'r') as f:
    B = parse_expr(f.read())

# print(A)
# print(B)

In [305]:
A_num = np.array(A.subs(subs_dict).evalf().tolist()).astype(np.float64)
B_num = np.array(B.subs(subs_dict).evalf().tolist()).astype(np.float64)
C_num = np.array([
    [1, 0, 0, 0],   # x from encoder
    [0, 1, 0, 0],   # ẋ
    [0, 0, 1, 0],   # θ
    [0, 0, 0, 1]    # θ̇
])
D_num = np.zeros((4, 1))

sys = control.ss(A_num, B_num, C_num, D_num)

In [306]:
# discrete system initialization
dt = 0.001
sys_d = control.c2d(sys, dt)

In [307]:
os.makedirs('data/statespace_models', exist_ok=True)

# --- Save continuous system as CSV ---
pd.DataFrame(sys.A).to_csv('data/statespace_models/A_continuous.csv', index=False, header=False)
pd.DataFrame(sys.B).to_csv('data/statespace_models/B_continuous.csv', index=False, header=False)
pd.DataFrame(sys.C).to_csv('data/statespace_models/C_continuous.csv', index=False, header=False)
pd.DataFrame(sys.D).to_csv('data/statespace_models/D_continuous.csv', index=False, header=False)

# --- Save discrete system as CSV ---
pd.DataFrame(sys_d.A).to_csv('data/statespace_models/A_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.B).to_csv('data/statespace_models/B_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.C).to_csv('data/statespace_models/C_discrete.csv', index=False, header=False)
pd.DataFrame(sys_d.D).to_csv('data/statespace_models/D_discrete.csv', index=False, header=False)

# Save dt (time step) as simple text file
with open('data/statespace_models/dt_discrete.txt', 'w') as f:
    f.write(str(sys_d.dt))


## Controllability and Observability

In [308]:
ctrb_matrix = control.ctrb(A_num, B_num)
print("Rank of ctrb:", np.linalg.matrix_rank(ctrb_matrix))

Rank of ctrb: 4


In [309]:
eigvals, eigvecs = np.linalg.eig(A_num)
# for i, (eigval, eigvec) in enumerate(zip(eigvals, eigvecs.T)):
#     print(f"Eigenvalue {eigval:.4f}:")
#     print(f"Corresponding eigenvector:\n{eigvec}\n")

In [310]:
O = control.obsv(sys.A, sys.C) 
print(np.linalg.matrix_rank(O))

4


## LQR

In [311]:
Q = np.array([
    [1, 0, 0, 0],
    [0, 5, 0, 0],
    [0, 0, 10, 0],
    [0, 0, 0, 10]
])
R = np.array([[3]])
K, S, E = control.lqr(sys.A, sys.B, Q, R)
print("LQR gain K:", K)
print("LQR gain E:", E)

LQR gain K: [[0.57735027 1.91360849 5.35670085 2.08410568]]
LQR gain E: [-255.09460549+0.j           -0.45613351+0.j
   -1.14637831+0.85802351j   -1.14637831-0.85802351j]


In [312]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the K matrix
pd.DataFrame(K).to_csv('data/gain_matrices/K_Continuous.csv', index=False, header=False)

In [313]:
# A_cl = A_num - B_num @ K
# eigvals, eigvecs = np.linalg.eig(A_cl)
# for i, (eigval, eigvec) in enumerate(zip(eigvals, eigvecs.T)):
#     print(f"Eigenvalue {eigval:.4f}:")
#     print(f"Corresponding eigenvector:\n{eigvec}\n")
# sys_cl = control.ss(A_cl, B_num, C_num, D_num)

In [314]:
# # Gramian
# Wc = solve_continuous_lyapunov(A_cl, -B_num @ B_num.T)
# print("Controllability Gramian (via SciPy):\n", Wc)


## Discrete LQR

In [315]:
Q_d = np.array([
    [1, 0, 0, 0],
    [0, 5, 0, 0],
    [0, 0, 10, 0],
    [0, 0, 0, 10]
])
R_d = np.array([[3]])
K_d, S_d, E_d = control.dlqr(sys_d.A, sys_d.B, Q_d, R_d)
print("LQR gain K:", K_d)
print("LQR gain E:", E_d)

LQR gain K: [[0.50768952 1.68299079 4.74023272 1.83515909]]
LQR gain E: [0.77537442+0.j         0.99885391+0.00085704j 0.99885391-0.00085704j
 0.99954397+0.j        ]


In [316]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the K matrix
pd.DataFrame(K_d).to_csv('data/gain_matrices/K_Discrete.csv', index=False, header=False)

## LQG

In [317]:
x_std = 0.01      # encoder error in meters
dx_std = 0.01       # IMU-integrated linear velocity (m/s)
theta_std = 0.1    # integrated angle error (rad)
dtheta_std = 0.1  # IMU gyro noise (rad/s)

# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn = np.diag([
    (x_std),
    (dx_std),
    (theta_std),
    (dtheta_std)
])
G = np.eye(4)
# Qn = np.diag([1e6, 1e6, 1e6, 1e6])      # increase process noise → fast observer
Qn = np.diag([
    1,   # x
    1,   # dx (e.g., friction force uncertainty)
    1,   # theta (e.g., small external torque)
    1    # dtheta (e.g., fast unmodeled torque effect)
])
L, P, E = control.lqe(sys.A, G, sys.C, Qn, Rn)
print(L)

[[ 1.00373935e+01  4.99584421e-01 -1.47061347e-03 -5.42290545e-03]
 [ 4.99584421e-01  1.00198064e+01 -6.55256909e-02 -1.95259160e-01]
 [-1.47061347e-02 -6.55256909e-01  1.62467127e+00  3.88400511e+00]
 [-5.42290545e-02 -1.95259160e+00  3.88400511e+00  1.54299489e+01]]


In [318]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the L matrix
pd.DataFrame(L).to_csv('data/gain_matrices/L_Continuous.csv', index=False, header=False)

## Discrete LQG

In [319]:
x_std_d = 0.01      # encoder error in meters
dx_std_d = 0.01       # IMU-integrated linear velocity (m/s)
theta_std_d = 0.1    # integrated angle error (rad)
dtheta_std_d = 0.1  # IMU gyro noise (rad/s)


# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn_d = np.diag([
    (x_std_d),
    (dx_std_d),
    (theta_std_d)*50,
    (dtheta_std_d)*50
])
G_d = np.eye(4)
# Qn = np.diag([1e6, 1e6, 1e6, 1e6])      # increase process noise → fast observer
Qn_d = np.diag([
    1,   # x
    1,   # dx (e.g., friction force uncertainty)
    1,   # theta (e.g., small external torque)
    1    # dtheta (e.g., fast unmodeled torque effect)
])
L_d, P_d, E_d = control.dlqe(sys_d.A, G_d, sys_d.C, Qn, Rn)
print(L_d)

[[ 9.90195136e-01  9.90290341e-04 -3.75002541e-07 -2.37685950e-10]
 [ 9.52046546e-08  9.90195142e-01 -7.48785859e-04 -5.63992803e-07]
 [-3.02575120e-09 -6.16111474e-06  9.16094037e-01  1.12629095e-03]
 [-1.84056931e-10 -3.75531672e-07  2.89321838e-02  9.16106888e-01]]


In [320]:
# Ensure the folder exists
os.makedirs('data/gain_matrices', exist_ok=True)

# Save the L matrix
pd.DataFrame(L_d).to_csv('data/gain_matrices/L_Discrete.csv', index=False, header=False)

# Nonlinear Controller Design

In [321]:
with open('../dynamics/output/A_func.txt', 'r') as f:
    A_func = parse_expr(f.read())
A_func_num = A_func.subs(subs_dict)
display(A_func_num)
# print(A_func_num)

Matrix([
[0, 1,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              0,                                                                                                                    0],
[0, 0,                   0.358623393996421*tau*sin(theta(t))*cos(

In [322]:
with open('../dynamics/output/xdd_solution.txt', 'r') as f:
    xdd_solution = parse_expr(f.read())

with open('../dynamics/output/thetadd_solution.txt', 'r') as f:
    thetadd_solution = parse_expr(f.read())

xdd_partial = xdd_solution.subs(subs_dict)
thetadd_partial = thetadd_solution.subs(subs_dict)
display(xdd_partial)
display(thetadd_partial)

-0.000118584*tau*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 0.000669876*tau/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 3.83193997776e-5*sin(theta(t))*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 1.3239429264e-6*sin(theta(t))*Derivative(theta(t), t)**2/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2)

0.0019764*tau*cos(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 0.0045524*tau/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) - 3.90615696e-6*sin(theta(t))*cos(theta(t))*Derivative(theta(t), t)**2/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2) + 0.00147106890936*sin(theta(t))/(5.082572504e-5 - 3.90615696e-6*cos(theta(t))**2)

## Extended Kalman Filter

<img src="Diagrams/EKF/EKF.png" alt="Alt text" width="600" height="500"/>

In [323]:
Qn_nonlinear = np.diag([
    1,   # x
    1,   # dx (e.g., friction force uncertainty)
    1,   # theta (e.g., small external torque)
    1    # dtheta (e.g., fast unmodeled torque effect)
])

In [324]:
x_std_nonlinear = 0.01      # encoder error in meters
dx_std_nonlinear = 0.01       # IMU-integrated linear velocity (m/s)
theta_std_nonlinear = 0.1    # integrated angle error (rad)
dtheta_std_nonlinear = 0.1  # IMU gyro noise (rad/s)


# Rn = np.diag([1e-6, 1e-6, 1e-6, 1e-6])  # reduce measurement noise → trust sensors
Rn_nonlinear = np.diag([
    (x_std_nonlinear),
    (dx_std_nonlinear),
    (theta_std_nonlinear)*50,
    (theta_std_nonlinear)*50
])

In [325]:
# Ensure the folder exists
os.makedirs('data/nonlinear_matrices', exist_ok=True)

# Save the Qn matrix
pd.DataFrame(Qn_nonlinear).to_csv('data/nonlinear_matrices/Qn.csv', index=False, header=False)
# Save the Rn matrix
pd.DataFrame(Rn_nonlinear).to_csv('data/nonlinear_matrices/Rn.csv', index=False, header=False)